In [25]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from numpy import concatenate
from math import sqrt

In [26]:
train_data = pd.read_csv('train_data.csv', parse_dates = [[1, 2]],
                   index_col = 0)
train_data.drop('Unnamed: 0', axis = 1, inplace=True)
train_data.drop('Volume', axis = 1, inplace=True)
train_data.index.name = 'date'
print(train_data.shape)
train_data.head()

(430039, 6)


,MidPrice,LastPrice,BidPrice1,BidVolume1,AskPrice1,AskVolume1
date,,,,,,
2018-06-01 09:30:01,3.7865,3.786,3.786,21500.0,3.787,12900.0
2018-06-01 09:30:04,3.7835,3.784,3.783,44200.0,3.784,119200.0
2018-06-01 09:30:07,3.7835,3.784,3.783,42700.0,3.784,61100.0
2018-06-01 09:30:10,3.7845,3.785,3.784,145900.0,3.785,16900.0
2018-06-01 09:30:13,3.7835,3.784,3.783,347700.0,3.784,200100.0


In [27]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    for i in range(n_in - 1, -1, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(1, n_out + 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg
values = train_data.values
print(values.shape)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 10, 20)
for i in range(20):
    reframed.drop(reframed.columns[[61 + i, 62 + i, 63 + i, 64 + i, 65 + i]], 
                  axis=1, inplace=True)
reframed['label'] = reframed.iloc[:,60: ].mean(1)
reframed.drop(reframed.columns[[60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]], 
                  axis=1, inplace=True)
print(reframed.shape)
reframed.head()

(430039, 6)
(430010, 61)


,var1(t-9),var2(t-9),var3(t-9),var4(t-9),var5(t-9),var6(t-9),var1(t-8),var2(t-8),var3(t-8),var4(t-8),...,var4(t-1),var5(t-1),var6(t-1),var1(t-0),var2(t-0),var3(t-0),var4(t-0),var5(t-0),var6(t-0),label
9,0.864379,0.864379,0.864379,0.000695,0.864379,0.001424,0.859477,0.861111,0.859477,0.001429,...,0.000533,0.866013,0.009485,0.862745,0.861111,0.861111,0.000284,0.864379,0.008844,0.861846
10,0.859477,0.861111,0.859477,0.001429,0.859477,0.013162,0.859477,0.861111,0.859477,0.001381,...,0.000284,0.864379,0.008844,0.863562,0.861111,0.861111,0.001028,0.866013,0.009485,0.861806
11,0.859477,0.861111,0.859477,0.001381,0.859477,0.006746,0.861111,0.862745,0.861111,0.004718,...,0.001028,0.866013,0.009485,0.865196,0.861111,0.864379,0.000065,0.866013,0.009485,0.861601
12,0.861111,0.862745,0.861111,0.004718,0.861111,0.001865,0.859477,0.861111,0.859477,0.011243,...,0.000065,0.866013,0.009485,0.866013,0.866013,0.866013,0.000019,0.866013,0.000640,0.861520
13,0.859477,0.861111,0.859477,0.011243,0.859477,0.022095,0.859477,0.861111,0.859477,0.011402,...,0.000019,0.866013,0.000640,0.862745,0.864379,0.862745,0.001025,0.862745,0.001026,0.861601


In [28]:
values = reframed.values
n_train = 430000
train = values[:n_train, :]
test = values[n_train:, :] 
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(430000, 1, 60) (430000,) (10, 1, 60) (10,)


In [43]:
model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
history = model.fit(train_X, train_y, epochs=50, batch_size=72, 
                    validation_data=(test_X, test_y), verbose=2, shuffle=True)
#verbose = 2，每个epochs输出报告
#shuffle = False，不打乱数据集 

Train on 430000 samples, validate on 10 samples
Epoch 1/50
 - 55s - loss: 1.7120e-04 - val_loss: 1.9472e-05
Epoch 2/50
 - 44s - loss: 1.1310e-05 - val_loss: 3.0020e-07
Epoch 3/50
 - 46s - loss: 9.7615e-06 - val_loss: 4.2905e-07
Epoch 4/50
 - 43s - loss: 8.9350e-06 - val_loss: 7.3590e-06
Epoch 5/50
 - 43s - loss: 8.3710e-06 - val_loss: 4.8194e-06
Epoch 6/50
 - 43s - loss: 7.9942e-06 - val_loss: 3.5866e-06
Epoch 7/50
 - 46s - loss: 7.8100e-06 - val_loss: 1.5157e-06
Epoch 8/50
 - 41s - loss: 7.6508e-06 - val_loss: 1.4449e-06
Epoch 9/50
 - 41s - loss: 7.3942e-06 - val_loss: 3.6358e-06
Epoch 10/50
 - 42s - loss: 7.2967e-06 - val_loss: 1.8114e-07
Epoch 11/50
 - 44s - loss: 7.1263e-06 - val_loss: 1.4218e-05
Epoch 12/50
 - 43s - loss: 7.1189e-06 - val_loss: 2.5321e-06
Epoch 13/50
 - 41s - loss: 6.9943e-06 - val_loss: 8.6612e-08
Epoch 14/50
 - 41s - loss: 6.9198e-06 - val_loss: 3.3083e-06
Epoch 15/50
 - 45s - loss: 6.9231e-06 - val_loss: 3.4386e-06
Epoch 16/50
 - 43s - loss: 6.8396e-06 - val_lo

In [44]:
test_data = pd.read_csv('test_data.csv', parse_dates = [[1, 2]],
                   index_col = 0)
test_data.drop('Unnamed: 0', axis = 1, inplace=True)
test_data.drop('Volume', axis = 1, inplace=True)
test_data.index.name = 'date'
print(test_data.shape)
test_data.head()

(10000, 6)


,MidPrice,LastPrice,BidPrice1,BidVolume1,AskPrice1,AskVolume1
date,,,,,,
2018-10-08 09:30:01,3.4255,3.426,3.425,61500.0,3.426,135900.0
2018-10-08 09:30:01,3.4255,3.430,3.425,181600.0,3.426,135900.0
2018-10-08 09:30:03,3.4300,3.431,3.429,8200.0,3.431,664200.0
2018-10-08 09:30:05,3.4315,3.431,3.431,514600.0,3.432,15200.0
2018-10-08 09:30:07,3.4320,3.433,3.431,386900.0,3.433,158100.0


In [45]:
values = test_data.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 10, 0)
reframed.head()

,var1(t-9),var2(t-9),var3(t-9),var4(t-9),var5(t-9),var6(t-9),var1(t-8),var2(t-8),var3(t-8),var4(t-8),...,var3(t-1),var4(t-1),var5(t-1),var6(t-1),var1(t-0),var2(t-0),var3(t-0),var4(t-0),var5(t-0),var6(t-0)
9,0.981818,0.980447,0.983193,0.003000,0.980447,0.032694,0.981818,0.991620,0.983193,0.008869,...,0.991597,0.001950,0.988827,0.000794,0.993007,0.994413,0.991597,0.001950,0.994413,0.044177
10,0.981818,0.991620,0.983193,0.008869,0.980447,0.032694,0.994406,0.994413,0.994398,0.000396,...,0.991597,0.001950,0.994413,0.044177,0.995804,0.994413,0.997199,0.015842,0.994413,0.208441
11,0.994406,0.994413,0.994398,0.000396,0.994413,0.159882,0.998601,0.994413,1.000000,0.025141,...,0.997199,0.015842,0.994413,0.208441,0.995804,0.994413,0.997199,0.016121,0.994413,0.081157
12,0.998601,0.994413,1.000000,0.025141,0.997207,0.003635,1.000000,1.000000,1.000000,0.018901,...,0.997199,0.016121,0.994413,0.081157,0.995804,0.994413,0.997199,0.017225,0.994413,0.081060
13,1.000000,1.000000,1.000000,0.018901,1.000000,0.038038,0.979021,0.977654,0.980392,0.002346,...,0.997199,0.017225,0.994413,0.081060,0.995804,0.994413,0.997199,0.014410,0.994413,0.079832


In [46]:
n_test = 9991
test_values = reframed.values
test_values = test_values[:n_test, :]
test_values = test_values.reshape((test_values.shape[0], 1, test_values.shape[1]))

In [47]:
y_predict = model.predict(test_values, batch_size=72, verbose=2)
print(y_predict.shape)
print(test_values.shape)
y_predict = pd.DataFrame(y_predict)
for i in range(5):
    y_predict.insert(i + 1, i + 1, y_predict[0])
y_predict.head()
y_predict = scaler.inverse_transform(y_predict)

(9991, 1)
(9991, 1, 60)


In [48]:
results = list()
indexes = list()
for i in range(142, 1000):
    results.append(y_predict[i * 10][0])
    indexes.append(i + 1)
results = pd.DataFrame(results)
indexes = pd.DataFrame(indexes)
indexes.head()
results = pd.concat([indexes, results], 1)
results.columns = ['caseid', 'midprice']
results.head()

,caseid,midprice
0,143,3.356102
1,144,3.363789
2,145,3.354922
3,146,3.355313
4,147,3.351877


In [49]:
results.to_csv('results.csv', index = False)